In [ ]:
import pandas as pd
import numpy as np
import yaml
from pathlib import Path

from db_psql_model import DatabaseCursor
from utils import get_season, nfl_weeks_pull, game_keys_pull
from yahoo_query import league_season_data

pd.set_option("display.max_columns", 999)

# TODAY = np.datetime64("today", "D")
TODAY = np.datetime64("2021-09-28")
# TODAY = np.datetime64("2020-09-28")
# TODAY = np.datetime64("2019-09-28")
# TODAY = np.datetime64("2018-09-28")
# TODAY = np.datetime64("2017-09-28")
# TODAY = np.datetime64("2016-09-28")
# TODAY = np.datetime64("2015-09-28")
# TODAY = np.datetime64("2014-09-28")
# TODAY = np.datetime64("2013-09-28")
# TODAY = np.datetime64("2012-09-28")

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
NFL_WEEKS = nfl_weeks_pull()
GAME_KEYS = game_keys_pull(first="no")
# SEASON = get_season()
SEASON = TODAY.astype("datetime64[Y]").item().year
LEAGUE_ID = GAME_KEYS[GAME_KEYS["season"] == SEASON]["league_ID"].values[0]
GAME_ID = GAME_KEYS[GAME_KEYS["season"] == SEASON]["game_id"].values[0]
nfl_weeks_list = list(NFL_WEEKS["week"][NFL_WEEKS["game_id"] == GAME_ID])

try:
    with open(PATH) as file:
        credentials = yaml.load(file, Loader=yaml.FullLoader)

except Exception as error:
    print(error)

CONSUMER_KEY = credentials["YFPY_CONSUMER_KEY"]
CONSUMER_SECRET = credentials["YFPY_CONSUMER_SECRET"]

league = league_season_data(
    auth_dir=PATH.parent,
    league_id=LEAGUE_ID,
    game_id=GAME_ID,
    game_code="nfl",
    offline=False,
    all_output_as_json=False,
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    browser_callback=True,
)

# game_keys = league.all_game_keys()
# nfl_weeks = league.all_nfl_weeks()

# meta = league.metadata(first_time=first_time)
# settings, roster, stat_cat = league.set_roster_pos_stat_cat(first_time=first_time)
# draft = league.draft_results(first_time=first_time)
# teams = league.teams_and_standings(first_time="no")

league.matchups_by_week_regseason(first_time='yes', nfl_week=1)
for week in nfl_weeks_list[1:-1]:
    league.matchups_by_week_regseason(first_time='no', nfl_week=week)

# players = league.players_list(first_time="no")

league.team_roster_by_week(first_time='yes', nfl_week=1)
for week in nfl_weeks_list[1:-1]:
    league.team_roster_by_week(first_time='no', nfl_week=week)

In [ ]:
# cur = DatabaseCursor(PATH / "private.yaml", options="-c search_path=dev")
# df = pd.read_csv(PATH / "assests" / "game_keys.csv")
# cur.copy_table_to_postgres_new(df, "test", "no")

In [ ]:
# sql_query = """
# SELECT *
# FROM dev.test
# """
# db_cursor = DatabaseCursor(PATH / "private.yaml", options="-c search_path=dev")
# db_cursor.copy_data_from_postgres(sql_query)

In [16]:
import pandas as pd
import numpy as np
import logging
import yaml
from pathlib import Path
from yfpy import YahooFantasySportsQuery
from yfpy.utils import complex_json_handler, unpack_data
from yfpy import get_logger

from db_psql_model import DatabaseCursor

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
TEAMS_FILE = list(Path().cwd().parent.glob("**/teams.yaml"))[0]

with open(PATH) as file:
    credentials = yaml.load(file, Loader=yaml.FullLoader)

LOGGET = get_logger(__name__)
LOG_OUTPUT = False
logging.getLogger("yfpy.query").setLevel(level=logging.INFO)

consumer_key = credentials['YFPY_CONSUMER_KEY']
consumer_secret = credentials['YFPY_CONSUMER_SECRET']

yahoo_query = YahooFantasySportsQuery(
    auth_dir='',
    league_id='103661',
    game_id='406',
    game_code='nfl',
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
)

In [ ]:
data = yahoo_query.get_team_stats_by_week(1, 17)

TypeError: Object of type <class 'dict'> with value of {'team_points': {
  "coverage_type": "week",
  "total": 130.7,
  "week": 17
}, 'team_projected_points': {
  "coverage_type": "week",
  "total": 125.13,
  "week": 17
}} is not JSON serializable